In [1]:
import pandas as pd

import pickle

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479513631630414

In [10]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():
    mlflow.set_tag('developper', 'walid')
    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')

    alpha = 0.01

    mlflow.log_param('alpha', alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

In [12]:
import xgboost as xgb

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [26]:
params = {
    "learning_rate"	:0.053308491791199335,
    "max_depth"	:97,
    "min_child_weight"	:8.007099032513992,
    "objective"	:"reg:linear",
    "reg_alpha"	:0.11690883335960077,
    "reg_lambda"	:0.01174921364469295,
   "seed"	:42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2022/05/25 17:45:51 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '9c7e6b8d479f461a9092e4b0d224a09c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[17:45:51] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:20.24223
[1]	validation-rmse:19.34688
[2]	validation-rmse:18.50419
[3]	validation-rmse:17.71156
[4]	validation-rmse:16.96661
[5]	validation-rmse:16.26637
[6]	validation-rmse:15.60901
[7]	validation-rmse:14.99221
[8]	validation-rmse:14.41377
[9]	validation-rmse:13.87153
[10]	validation-rmse:13.36391
[11]	validation-rmse:12.88896
[12]	validation-rmse:12.44505
[13]	validation-rmse:12.03038
[14]	validation-rmse:11.64323
[15]	validation-rmse:11.28167
[16]	validation-rmse:10.94469
[17]	validation-rmse:10.63108
[18]	validation-rmse:10.33946
[19]	validation-rmse:10.06856
[20]	validation-rmse:9.81647
[21]	validation-rmse:9.58282
[22]	validation-rmse:9.36626
[23]	validation-rmse:9.16535
[24]	validation-rmse:8.97935
[25]	validation-rmse:8.80723
[26]	validation-rmse:8.64822
[27]	validation-rmse:8.50072
[28]	validation-rmse:8.36453
[29]	validation-rmse:8.23898
[

2022/05/25 17:48:52 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/05/25 17:48:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/wld/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [10]:
params = {
    "learning_rate"	:0.053308491791199335,
    "max_depth"	:97,
    "min_child_weight"	:8.007099032513992,
    "objective"	:"reg:linear",
    "reg_alpha"	:0.11690883335960077,
    "reg_lambda"	:0.01174921364469295,
   "seed"	:42
}

In [17]:
with mlflow.start_run():
    mlflow.set_tag('developper', 'walid')
    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')

    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

[11:55:44] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:20.24223
[1]	validation-rmse:19.34688
[2]	validation-rmse:18.50419
[3]	validation-rmse:17.71156
[4]	validation-rmse:16.96661
[5]	validation-rmse:16.26637
[6]	validation-rmse:15.60901
[7]	validation-rmse:14.99221
[8]	validation-rmse:14.41377
[9]	validation-rmse:13.87153
[10]	validation-rmse:13.36391
[11]	validation-rmse:12.88896
[12]	validation-rmse:12.44505
[13]	validation-rmse:12.03038
[14]	validation-rmse:11.64323
[15]	validation-rmse:11.28167
[16]	validation-rmse:10.94469
[17]	validation-rmse:10.63108
[18]	validation-rmse:10.33946
[19]	validation-rmse:10.06856
[20]	validation-rmse:9.81647
[21]	validation-rmse:9.58282
[22]	validation-rmse:9.36626
[23]	validation-rmse:9.16535
[24]	validation-rmse:8.97935
[25]	validation-rmse:8.80723
[26]	validation-rmse:8.64822
[27]	validation-rmse:8.50072
[28]	validation-rmse:8.36453
[29]	validation-rmse:8.23898
[

/home/wld/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [20]:
alpha = 0.1

In [25]:
with mlflow.start_run():
    mlflow.set_tag('developper', 'walid')
    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')
    mlflow.set_tag("model", "LinearRegressionLasso")
    mlflow.log_param('alpha', alpha)
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(lr, artifact_path='models_mlflow')

In [26]:
from mlflow.tracking import MlflowClient

ML_FLOW_TRACKING_URI = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri = ML_FLOW_TRACKING_URI)

In [30]:
client.list_artifacts('b4cccdafc30542e5ad6b8aad12801e6b')

[<FileInfo: file_size=None, is_dir=True, path='models_mlflow'>]

In [37]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [39]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = 1,
    filter_string="",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ['metric.rmse ASC']
    
)

In [42]:
runs

[<Run: data=<RunData: metrics={'rmse': 6.476742012409034}, params={'learning_rate': '0.07097336538904457',
  'max_depth': '91',
  'min_child_weight': '0.405555456388477',
  'objective': 'reg:linear',
  'reg_alpha': '0.3262431553108854',
  'reg_lambda': '0.0028187336743046355',
  'seed': '42'}, tags={'mlflow.source.name': '/home/wld/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'wld',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='./mlruns/1/3a679167f73843228c2c02ecdfad75c9/artifacts', end_time=1653493018768, experiment_id='1', lifecycle_stage='active', run_id='3a679167f73843228c2c02ecdfad75c9', run_uuid='3a679167f73843228c2c02ecdfad75c9', start_time=1653492743973, status='FINISHED', user_id='wld'>>,
 <Run: data=<RunData: metrics={'rmse': 6.530046194703382}, params={'learning_rate': '0.10895506239123798',
  'max_depth': '53',
  'min_child_weight': '6.026829541949242',
  'objective': 'reg:linear',
  'reg_alpha': '0.0

In [44]:
run_id = "9898d304870b43d1aac6f18e2a86a0c4"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regression')

Registered model 'nyc-taxi-regression' already exists. Creating a new version of this model...
2022/05/26 12:23:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regression, version 3
Created version '3' of model 'nyc-taxi-regression'.


<ModelVersion: creation_timestamp=1653560595181, current_stage='None', description=None, last_updated_timestamp=1653560595181, name='nyc-taxi-regression', run_id='9898d304870b43d1aac6f18e2a86a0c4', run_link=None, source='./mlruns/1/9898d304870b43d1aac6f18e2a86a0c4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [45]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1653559388826, description='', last_updated_timestamp=1653560595181, latest_versions=[<ModelVersion: creation_timestamp=1653559397162, current_stage='Staging', description='', last_updated_timestamp=1653559449878, name='nyc-taxi-regression', run_id='81e3b60750384f60b713ef98cdf289ac', run_link='', source='./mlruns/1/81e3b60750384f60b713ef98cdf289ac/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=2>,
  <ModelVersion: creation_timestamp=1653560595181, current_stage='None', description=None, last_updated_timestamp=1653560595181, name='nyc-taxi-regression', run_id='9898d304870b43d1aac6f18e2a86a0c4', run_link=None, source='./mlruns/1/9898d304870b43d1aac6f18e2a86a0c4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regression', tags={}>,
 <RegisteredModel: creation_timestamp=1653560571381, description=None, last_updated_timestamp=165356

In [48]:
model_name = 'nyc-taxi-regression'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


In [49]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage='Staging',
    archive_existing_versions=False)

<ModelVersion: creation_timestamp=1653560595181, current_stage='Staging', description=None, last_updated_timestamp=1653560927866, name='nyc-taxi-regression', run_id='9898d304870b43d1aac6f18e2a86a0c4', run_link=None, source='./mlruns/1/9898d304870b43d1aac6f18e2a86a0c4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [50]:
runs[0]

<Run: data=<RunData: metrics={'rmse': 6.476742012409034}, params={'learning_rate': '0.07097336538904457',
 'max_depth': '91',
 'min_child_weight': '0.405555456388477',
 'objective': 'reg:linear',
 'reg_alpha': '0.3262431553108854',
 'reg_lambda': '0.0028187336743046355',
 'seed': '42'}, tags={'mlflow.source.name': '/home/wld/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'wld',
 'model': 'xgboost'}>, info=<RunInfo: artifact_uri='./mlruns/1/3a679167f73843228c2c02ecdfad75c9/artifacts', end_time=1653493018768, experiment_id='1', lifecycle_stage='active', run_id='3a679167f73843228c2c02ecdfad75c9', run_uuid='3a679167f73843228c2c02ecdfad75c9', start_time=1653492743973, status='FINISHED', user_id='wld'>>